# Annotate fragment using nonRedundant TF archetype

**Set environment**

In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



**Input**

In [4]:
ls -d ${FD_RES}/count_fragment/Input?_20x

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x


In [6]:
ls -d ${FD_RES}/count_fragment/Input?_20x/chr17.bed.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/chr17.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/chr17.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/chr17.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x/chr17.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/chr17.bed.gz


In [15]:
ls -d ${FD_RES}/count_fragment/Input?_20x/chr1.bed.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/chr1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/chr1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/chr1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x/chr1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/chr1.bed.gz


## Annotate fragments for each chromosome

**TEST: Loop**

In [7]:
### Loop chromosomes
CHROMS=($(seq 1 22) X Y)
for idx in {0..23}; do
    echo "Index: $idx; Chromosome ${CHROMS[$idx]}"
done

Index: 0; Chromosome 1
Index: 1; Chromosome 2
Index: 2; Chromosome 3
Index: 3; Chromosome 4
Index: 4; Chromosome 5
Index: 5; Chromosome 6
Index: 6; Chromosome 7
Index: 7; Chromosome 8
Index: 8; Chromosome 9
Index: 9; Chromosome 10
Index: 10; Chromosome 11
Index: 11; Chromosome 12
Index: 12; Chromosome 13
Index: 13; Chromosome 14
Index: 14; Chromosome 15
Index: 15; Chromosome 16
Index: 16; Chromosome 17
Index: 17; Chromosome 18
Index: 18; Chromosome 19
Index: 19; Chromosome 20
Index: 20; Chromosome 21
Index: 21; Chromosome 22
Index: 22; Chromosome X
Index: 23; Chromosome Y


In [19]:
### Loop folders
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/Input?_20x/))
for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    echo "SAMPLE: ${SAMPLE}; FD_BED: ${FD_BED}"
done

SAMPLE: Input1_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/
SAMPLE: Input2_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/
SAMPLE: Input3_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/
SAMPLE: Input4_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x/
SAMPLE: Input5_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/


**RUN**

In [9]:
ls -d ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log
all


In [23]:
### set env
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --array 0-23 \
    --mem 20G \
    -o ${FD_LOG}/annotate_fragment_chrom_input20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
#FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/Input?_20x/))
FN_BED=${TARGET}.bed.gz

### motif annotation
FD_MTF=${FD_ANN}/motif_cluster_jvierstra/hg38_archetype_motifs_v1
FN_MTF=${CHROM}_rm_mouse_merge.bed.gz

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; annotate fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/annotation_fragment/${SAMPLE}
    FN_OUT=${TARGET}.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3 
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### annotation using intersect
    bedtools intersect -a ${FD_BED}/${FN_BED} -b ${FD_MTF}/${FN_MTF} -wo |\
        gzip -c > ${FD_OUT}/${FN_OUT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    zcat ${FD_OUT}/${FN_OUT} | head -n 3
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26210963


**CHECK**

In [25]:
tail ${FD_LOG}/annotate_fragment_chrom_input20x.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/annotate_fragment_chrom_input20x.0.txt <==
chr1	13868	14793	1

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input5_20x/chr1.bed.gz
chr1	12225	13098	1	chr1	12209	12228	ZFN121	4.4991	3
chr1	12225	13098	1	chr1	12213	12227	NR/3	7.5332	2
chr1	12225	13098	1	chr1	12213	12231	REST/NRSF	4.8498	6

Done!
Run Time: 13 hours 25 minutes and 30 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/annotate_fragment_chrom_input20x.10.txt <==
chr11	102297	103363	2

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input5_20x/chr11.bed.gz
chr11	72984	73929	1	chr11	72984	72994	ZNF384/2	8.896275	10
chr11	72984	73929	1	chr11	72985	72997	FOX/5	10.0476	12
chr11	72984	73929	1	chr11	72987	72996	FEZF1	8.2312	9

Done!
Run Time: 7 hours 35 minutes and 42 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/l

In [24]:
cat ${FD_LOG}/annotate_fragment_chrom_input20x.0.txt

Slurm Array Index:  0
Time Stamp:         10-01-21+17:55:26

RUN: loop through samples; annotate fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x//chr1.bed.gz
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1_20x/chr1.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x//chr1.bed.gz
chr1	13802	14757	1
chr1	13802	14759	2
chr1	13868	14793	2

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1_20x/chr1.bed.gz
chr1	13802	14757	1	chr1	13790	13810	ZNF335	6.8996	8
chr1	13802	14757	1	chr1	13792	13806	NR/3	8.1207	4
chr1	13802	14757	1	chr1	13794	13814	GC-tract	6.2461	12

++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x//chr1.bed.gz
Ou